In [12]:
%matplotlib inline
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

TRAIN DATA 

In [24]:
movies_train_df = pd.read_csv(Path('movie_data_frame12.csv'))
movies_train_df

,movie_id,movie_rating,release_date,run_time,meta_score,imdb_rating,box_office,director,writer,actor,genre_name
0,1,R,1980-06-13,146.0,0.0,8.4,46998772.0,Stanley Kubrick,Stephen King,Jack Nicholson,Drama
1,2,R,1980-07-02,104.0,0.0,5.8,58853106.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,Adventure
2,3,PG,1980-06-20,124.0,0.0,8.7,538375067.0,Irvin Kershner,Leigh Brackett,Mark Hamill,Action
3,4,PG,1980-07-02,88.0,0.0,7.7,83453539.0,Jim Abrahams,Jim Abrahams,Robert Hays,Comedy
4,5,R,1980-07-25,98.0,0.0,7.3,39846344.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,Comedy
...,...,...,...,...,...,...,...,...,...,...,...
7925,7926,PG-13,2022-01-30,65.0,0.0,0.0,793669.0,Peter Jackson,0,The Beatles,Music
7926,7927,PG,2022-02-03,102.0,33.0,5.8,0.0,Ray Giarratana,Ray Giarratana,Queen Latifah,Family
7927,7928,NaN,1900-01-01,0.0,0.0,0.0,0.0,Katy Maudlin,Morgan Rose,Keegan Joyce,Short
7928,7929,NaN,2022-01-07,88.0,0.0,0.0,0.0,Jeff Joslin,Ciera Danielle,Jeff Joslin,Biography


In [25]:
movies_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7930 entries, 0 to 7929
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   movie_id      7930 non-null   int64  
 1   movie_rating  7772 non-null   object 
 2   release_date  7930 non-null   object 
 3   run_time      7930 non-null   float64
 4   meta_score    7930 non-null   float64
 5   imdb_rating   7930 non-null   float64
 6   box_office    7930 non-null   float64
 7   director      7930 non-null   object 
 8   writer        7930 non-null   object 
 9   actor         7930 non-null   object 
 10  genre_name    7763 non-null   object 
dtypes: float64(4), int64(1), object(6)
memory usage: 681.6+ KB


In [27]:
movies_train_df['box_office'] = movies_train_df['box_office'].astype(str)

In [28]:
movies_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7930 entries, 0 to 7929
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   movie_id      7930 non-null   int64  
 1   movie_rating  7772 non-null   object 
 2   release_date  7930 non-null   object 
 3   run_time      7930 non-null   float64
 4   meta_score    7930 non-null   float64
 5   imdb_rating   7930 non-null   float64
 6   box_office    7930 non-null   object 
 7   director      7930 non-null   object 
 8   writer        7930 non-null   object 
 9   actor         7930 non-null   object 
 10  genre_name    7763 non-null   object 
dtypes: float64(3), int64(1), object(7)
memory usage: 681.6+ KB


In [31]:
movies_train_df['label']=''
for index,row in movies_train_df.iterrows():
    if (row['box_office']<='100000'):
        movies_train_df.loc[index,'label']=0
    elif (row['box_office']>'100000') & (row['box_office']<='500000'):
        movies_train_df.loc[index, 'label']=1
    elif (row['box_office']>'500000') & (row['box_office']<='1000000'):
        movies_train_df.loc[index, 'label']=2
    elif (row['box_office']>'1000000') & (row['box_office']<='5000000'):
        movies_train_df.loc[index, 'label']=3
    elif (row['box_office']>'5000000') & (row['box_office']<='1000000000'):
        movies_train_df.loc[index, 'label']=4  
    elif (row['box_office']>'1000000000') & (row['box_office']<='5000000000'):
        movies_train_df.loc[index, 'label']=5
    elif (row['box_office']>'5000000000'):
         movies_train_df.loc[index, 'label']=6
    else:
        movies_train_df.loc[index, 'label']=0
        
#labels_6 =["0-100,000","100,000-500,000","500,000-1,000,000","1,000,000-5,000,000",
 #          "5,000,000,000-1,000,000,000","1,000,000,000-5,000,000,000"]

In [32]:
movies_train_df['label'].value_counts()

1    5534
6    2153
0     240
5       2
3       1
Name: label, dtype: int64

In [33]:
# movies_train_df['score'] = movies_train_df['score'].replace(np.nan, 0)

In [34]:
movies_train_df['label'].value_counts()

1    5534
6    2153
0     240
5       2
3       1
Name: label, dtype: int64

In [36]:
#movies_train_df = movies_train_df.drop('budget', axis = 1)

In [37]:
movies_train_df = movies_train_df.dropna(axis = 0, how='any')
movies_train_df

,movie_id,movie_rating,release_date,run_time,meta_score,imdb_rating,box_office,director,writer,actor,genre_name,label
0,1,R,1980-06-13,146.0,0.0,8.4,46998772.0,Stanley Kubrick,Stephen King,Jack Nicholson,Drama,1
1,2,R,1980-07-02,104.0,0.0,5.8,58853106.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,Adventure,6
2,3,PG,1980-06-20,124.0,0.0,8.7,538375067.0,Irvin Kershner,Leigh Brackett,Mark Hamill,Action,6
3,4,PG,1980-07-02,88.0,0.0,7.7,83453539.0,Jim Abrahams,Jim Abrahams,Robert Hays,Comedy,6
4,5,R,1980-07-25,98.0,0.0,7.3,39846344.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,Comedy,1
...,...,...,...,...,...,...,...,...,...,...,...,...
7922,7923,PG-13,2022-01-21,134.0,32.0,6.9,0.0,D.J. Caruso,Francine Rivers,Tom Lewis,Drama,0
7923,7924,PG-13,2022-02-11,108.0,0.0,0.0,1471504.0,Mark Williams,Nick May,Taylor John Smith,Action,1
7924,7925,PG,2022-01-21,90.0,30.0,5.1,391252.0,Sean McNamara,Vonda N. McIntyre,William Hurt,Action,1
7925,7926,PG-13,2022-01-30,65.0,0.0,0.0,793669.0,Peter Jackson,0,The Beatles,Music,6


In [39]:
y = movies_train_df['label'].astype('int')
X = movies_train_df.drop(['label', 'box_office'], axis = 1)

In [40]:
y.value_counts()

1    5343
6    2074
0     189
5       2
Name: label, dtype: int64

In [41]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
transformed = lb_make.fit_transform(X['movie_rating'])
X['movie_rating'] = transformed

transformed = lb_make.fit_transform(X['release_date'])
X['release_date'] = transformed

transformed = lb_make.fit_transform(X['director'])
X['director'] = transformed

transformed = lb_make.fit_transform(X['writer'])
X['writer'] = transformed

transformed = lb_make.fit_transform(X['actor'])
X['actor'] = transformed

transformed = lb_make.fit_transform(X['genre_name'])
X['genre_name'] = transformed

X

,movie_id,movie_rating,release_date,run_time,meta_score,imdb_rating,director,writer,actor,genre_name
0,1,6,17,146.0,0.0,8.4,2595,4003,1020,7
1,2,6,21,104.0,0.0,5.8,2276,1612,310,1
2,3,4,18,124.0,0.0,8.7,1103,2541,1725,0
3,4,4,21,88.0,0.0,7.7,1296,1977,2233,4
4,5,6,25,98.0,0.0,7.3,1049,512,392,4
...,...,...,...,...,...,...,...,...,...,...
7922,7923,5,2969,134.0,32.0,6.9,536,1341,2650,7
7923,7924,5,2973,108.0,0.0,0.0,1823,3147,2566,0
7924,7925,4,2969,90.0,30.0,5.1,2533,4379,2781,0
7925,7926,5,2970,65.0,0.0,0.0,2198,1,2593,12


In [42]:
y

0       1
1       6
2       6
3       6
4       1
       ..
7922    0
7923    1
7924    1
7925    6
7926    0
Name: label, Length: 7608, dtype: int64

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [44]:
y_train

996     0
6713    6
7393    1
3320    6
1668    1
       ..
915     1
5297    6
4049    1
238     1
5262    1
Name: label, Length: 5706, dtype: int64

In [45]:
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_scaled

array([[-1.26852591, -0.99137721, -1.16295002, ...,  1.60521762,
         0.69000505, -0.94112205],
       [ 1.25814788,  0.66556432,  1.28648549, ...,  1.27632031,
         1.08890038, -1.26707303],
       [ 1.5586793 ,  0.66556432,  1.5597009 , ...,  1.16490731,
         0.7448989 , -1.26707303],
       ...,
       [ 0.08077181,  0.66556432,  0.09502874, ...,  0.43309173,
         1.52073201,  0.36268187],
       [-1.60353006,  0.66556432, -1.59059153, ..., -1.32128159,
         1.17551068,  0.03673089],
       [ 0.61686684, -0.16290645,  0.64027166, ..., -0.59099222,
         1.68297384,  1.01458383]])

In [46]:
classifier = LogisticRegression()
classifier

LogisticRegression()

In [47]:
y_train

996     0
6713    6
7393    1
3320    6
1668    1
       ..
915     1
5297    6
4049    1
238     1
5262    1
Name: label, Length: 5706, dtype: int64

In [48]:
classifier.fit(X_train, y_train)

/Users/rebeccamelo/opt/anaconda3/envs/PythonData/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [49]:
y_predict = classifier.predict(X_test)

In [50]:
y_predict

array([1, 1, 1, ..., 1, 1, 1])

In [51]:
print(y_test - y_predict)

4862    0
6181    5
4349    0
3225    5
2735    0
       ..
531     0
5343    0
4461    0
6412    5
6910    0
Name: label, Length: 1902, dtype: int64


In [52]:
classifier.score(X_train, y_train)

0.7010164738871364

In [53]:
classifier.score(X_test, y_test)

0.7060988433228181

In [54]:
classifier.score(X_test, y_predict)

1.0

In [56]:
y_pred = pd.DataFrame(classifier.predict(X_test), index=X_test.index, columns=['Predictions'])
y_pred_df = y_pred.merge(y_test, how='outer', left_index=True, right_index=True)
y_pred_df

,Predictions,label
4862,1,1
6181,1,6
4349,1,1
3225,1,6
2735,1,1
...,...,...
531,1,1
5343,1,1
4461,1,1
6412,1,6


In [57]:
# save csv so we don't have to run the loop again
y_pred_df.to_csv("predictions_retest.csv", index=False)

TEST DATA START

In [40]:
movies_test_df = movies_test_df = pd.read_csv(Path('upcoming_df_2022.csv'))
movies_test_df

,movie_id,title,movie_rating,release_date,runtime,genre_name,Director,Writer,Actors,Language,Poster,metascore,imdb_rating
0,0,Studio 666,R,2/25/2022,0,"Comedy, Horror, Music",BJ McDonnell,"Dave Grohl, Jeff Buhler, Rebecca Hughes","Jenna Ortega, Will Forte, Leslie Grossman",English,https://m.media-amazon.com/images/M/MV5BMmU5Nj...,0,0.0
1,1,Butter,0,1/31/2022,19,"Short, Comedy, Drama",Anita Lee,Madeleine Withington,"Amber Adams, Matthew Backer, Jenna Bell",English,0,0,0.0
2,2,The Batman,PG-13,3/4/2022,175,"Action, Crime, Drama",Matt Reeves,"Matt Reeves, Peter Craig, Bob Kane","Barry Keoghan, Zoë Kravitz, Robert Pattinson",English,https://m.media-amazon.com/images/M/MV5BYTExZT...,0,0.0
3,4,He Was Asking for It: A Noir Film,0,4/30/2022,0,"Short, Crime",Rocky Karlage,"Rocky Karlage, Staci Layne Wilson","Rocky Karlage, Patrick Lamba, Charlie Jiin",English,https://m.media-amazon.com/images/M/MV5BMjA5NG...,0,0.0
4,5,X,R,3/18/2022,0,Horror,Ti West,Ti West,"Jenna Ortega, Mia Goth, Brittany Snow",English,https://m.media-amazon.com/images/M/MV5BYjI1OW...,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,75,Strange World,0,11/23/2022,0,"Animation, Action, Adventure",Don Hall,Qui Nguyen,Alan Tudyk,English,https://m.media-amazon.com/images/M/MV5BMDNjOT...,0,0.0
71,76,Avatar 2,0,12/16/2022,0,"Action, Adventure, Sci-Fi",James Cameron,"James Cameron, Josh Friedman","Kate Winslet, Zoe Saldana, Michelle Yeoh",English,https://m.media-amazon.com/images/M/MV5BNmM1Nm...,0,0.0
72,77,Aquaman and the Lost Kingdom,0,12/16/2022,0,"Action, Adventure, Fantasy",James Wan,"David Leslie Johnson-McGoldrick, Paul Norris, ...","Temuera Morrison, Jason Momoa, Nicole Kidman",English,https://m.media-amazon.com/images/M/MV5BM2QyYT...,0,0.0
73,78,Untitled Illumination Entertainment Project,0,12/21/2022,0,"Animation, Adventure, Comedy","Aaron Horvath, Michael Jelenic","Matthew Fogel, Shigeru Miyamoto","Anya Taylor-Joy, Chris Pratt, Seth Rogen",English,https://m.media-amazon.com/images/M/MV5BODMzNm...,0,0.0


In [41]:
movies_test_df['label']=''
for index,row in movies_test_df.iterrows():
    if (row['imdb_rating']<=5):
        movies_test_df.loc[index,'label']=0
    elif (row['imdb_rating']>5) & (row['imdb_rating']<=7):
        movies_test_df.loc[index, 'label']=1
    elif (row['imdb_rating']>7):
         movies_test_df.loc[index, 'label']=2

In [42]:
movies_test_df['label'].value_counts()

0    68
1     5
2     2
Name: label, dtype: int64

In [43]:
movies_test_df = movies_test_df.dropna(axis = 0, how='any')
movies_test_df

,movie_id,title,movie_rating,release_date,runtime,genre_name,Director,Writer,Actors,Language,Poster,metascore,imdb_rating,label
0,0,Studio 666,R,2/25/2022,0,"Comedy, Horror, Music",BJ McDonnell,"Dave Grohl, Jeff Buhler, Rebecca Hughes","Jenna Ortega, Will Forte, Leslie Grossman",English,https://m.media-amazon.com/images/M/MV5BMmU5Nj...,0,0.0,0
1,1,Butter,0,1/31/2022,19,"Short, Comedy, Drama",Anita Lee,Madeleine Withington,"Amber Adams, Matthew Backer, Jenna Bell",English,0,0,0.0,0
2,2,The Batman,PG-13,3/4/2022,175,"Action, Crime, Drama",Matt Reeves,"Matt Reeves, Peter Craig, Bob Kane","Barry Keoghan, Zoë Kravitz, Robert Pattinson",English,https://m.media-amazon.com/images/M/MV5BYTExZT...,0,0.0,0
3,4,He Was Asking for It: A Noir Film,0,4/30/2022,0,"Short, Crime",Rocky Karlage,"Rocky Karlage, Staci Layne Wilson","Rocky Karlage, Patrick Lamba, Charlie Jiin",English,https://m.media-amazon.com/images/M/MV5BMjA5NG...,0,0.0,0
4,5,X,R,3/18/2022,0,Horror,Ti West,Ti West,"Jenna Ortega, Mia Goth, Brittany Snow",English,https://m.media-amazon.com/images/M/MV5BYjI1OW...,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,75,Strange World,0,11/23/2022,0,"Animation, Action, Adventure",Don Hall,Qui Nguyen,Alan Tudyk,English,https://m.media-amazon.com/images/M/MV5BMDNjOT...,0,0.0,0
71,76,Avatar 2,0,12/16/2022,0,"Action, Adventure, Sci-Fi",James Cameron,"James Cameron, Josh Friedman","Kate Winslet, Zoe Saldana, Michelle Yeoh",English,https://m.media-amazon.com/images/M/MV5BNmM1Nm...,0,0.0,0
72,77,Aquaman and the Lost Kingdom,0,12/16/2022,0,"Action, Adventure, Fantasy",James Wan,"David Leslie Johnson-McGoldrick, Paul Norris, ...","Temuera Morrison, Jason Momoa, Nicole Kidman",English,https://m.media-amazon.com/images/M/MV5BM2QyYT...,0,0.0,0
73,78,Untitled Illumination Entertainment Project,0,12/21/2022,0,"Animation, Adventure, Comedy","Aaron Horvath, Michael Jelenic","Matthew Fogel, Shigeru Miyamoto","Anya Taylor-Joy, Chris Pratt, Seth Rogen",English,https://m.media-amazon.com/images/M/MV5BODMzNm...,0,0.0,0


In [44]:
movies_test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75 entries, 0 to 74
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   movie_id      75 non-null     int64  
 1   title         75 non-null     object 
 2   movie_rating  75 non-null     object 
 3   release_date  75 non-null     object 
 4   runtime       75 non-null     int64  
 5   genre_name    75 non-null     object 
 6   Director      75 non-null     object 
 7   Writer        75 non-null     object 
 8   Actors        75 non-null     object 
 9   Language      75 non-null     object 
 10  Poster        75 non-null     object 
 11  metascore     75 non-null     int64  
 12  imdb_rating   75 non-null     float64
 13  label         75 non-null     object 
dtypes: float64(1), int64(3), object(10)
memory usage: 8.8+ KB


In [45]:
y = movies_test_df['label'].astype('int')
X = movies_test_df.drop('label', axis = 1)
# X.info()

In [46]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75 entries, 0 to 74
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   movie_id      75 non-null     int64  
 1   title         75 non-null     object 
 2   movie_rating  75 non-null     object 
 3   release_date  75 non-null     object 
 4   runtime       75 non-null     int64  
 5   genre_name    75 non-null     object 
 6   Director      75 non-null     object 
 7   Writer        75 non-null     object 
 8   Actors        75 non-null     object 
 9   Language      75 non-null     object 
 10  Poster        75 non-null     object 
 11  metascore     75 non-null     int64  
 12  imdb_rating   75 non-null     float64
dtypes: float64(1), int64(3), object(9)
memory usage: 8.2+ KB


In [47]:
y.value_counts()

0    68
1     5
2     2
Name: label, dtype: int64

In [48]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
transformed = lb_make.fit_transform(X['title'])
X['title'] = transformed

transformed = lb_make.fit_transform(X['movie_rating'])
X['movie_rating'] = transformed

transformed = lb_make.fit_transform(X['genre_name'])
X['genre_name'] = transformed

transformed = lb_make.fit_transform(X['Director'])
X['Director'] = transformed

transformed = lb_make.fit_transform(X['Writer'])
X['Writer'] = transformed

transformed = lb_make.fit_transform(X['Actors'])
X['Actors'] = transformed

transformed = lb_make.fit_transform(X['Language'])
X['Language'] = transformed

transformed = lb_make.fit_transform(X['Poster'])
X['Poster'] = transformed

transformed = lb_make.fit_transform(X['release_date'])
X['release_date'] = transformed

X

,movie_id,title,movie_rating,release_date,runtime,genre_name,Director,Writer,Actors,Language,Poster,metascore,imdb_rating
0,0,45,3,16,0,18,7,17,35,1,19,0,0.0
1,1,11,0,2,19,34,5,49,3,1,0,0,0.0
2,2,47,2,19,175,7,48,51,8,1,45,0,0.0
3,4,25,0,26,0,35,60,67,59,1,13,0,0.0
4,5,73,3,17,0,32,73,71,34,1,50,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,75,44,0,11,0,14,20,63,1,1,2,0,0.0
71,76,5,0,13,0,3,31,34,43,1,32,0,0.0
72,77,3,0,13,0,2,32,18,68,1,1,0,0.0
73,78,71,0,14,0,16,0,53,5,1,34,0,0.0


In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [50]:
classifier = LogisticRegression()
classifier

LogisticRegression()

In [51]:
classifier.fit(X_train, y_train)

C:\Users\mgian\anaconda3\envs\PythonData\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [52]:
classifier.predict

<bound method LinearClassifierMixin.predict of LogisticRegression()>

In [54]:
y_predict = classifier.predict(X_test)
y_predict

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0])

In [55]:
classifier.score(X_train, y_train)

1.0

In [56]:
classifier.score(X_test, y_test)

0.8947368421052632

In [57]:
classifier.score(X_test, y_predict)

1.0